In [1]:
import numpy as np
from itertools import groupby

In [2]:
sample_input = open('sample-input.txt')
puzzles = [line.strip() for line in [line for line in sample_input.read().splitlines() if len(line)!=0] if line[0]!='#']
puzzles

['BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.',
 '..I...BBI.K.GHAAKLGHDDKLG..JEEFF.J..',
 'JBBCCCJDD..MJAAL.MFFKL.N..KGGN.HH...',
 'BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII J0 B4',
 'IJBBCCIJDDL.IJAAL.EEK.L...KFF..GGHH. F0 G6',
 'BB.G.HE..G.HEAAG.I..FCCIDDF..I..F...']

In [3]:
def create_grid(puzzle:str):
    return np.array(list(puzzle), dtype=str).reshape(6,6)

In [4]:
puzzle_grid = create_grid(puzzles[0])
print(puzzle_grid)

[['B' 'B' 'I' 'J' '.' '.']
 ['.' '.' 'I' 'J' 'C' 'C']
 ['.' '.' 'I' 'A' 'A' 'M']
 ['G' 'D' 'D' 'K' '.' 'M']
 ['G' 'H' '.' 'K' 'L' '.']
 ['G' 'H' 'F' 'F' 'L' '.']]


In [5]:
class Car():
    
    def __init__(self, is_horizontal:bool, letter:str, car_length:int, arr_indices):
        self.horizontal = is_horizontal
        self.letter = letter
        self.car_length = car_length
        self.arr_indices = arr_indices
    
    def update_arr_indices(self, amount:int):
        if self.horizontal:
            self.arr_indices = self.arr_indices[0], self.arr_indices[1] + amount
        else:
            self.arr_indices = self.arr_indices[0] + amount, self.arr_indices[1]
    
    def __str__(self):
        return "{direction} car '{letter}' with a length of {length}. Indices i:{indices_i} and j:{indices_j}".format(
            direction='Horizontal' if self.horizontal else 'Vertical',
            letter=self.letter,
            length=self.car_length,
            indices_i=self.arr_indices[0],
            indices_j=self.arr_indices[1])

In [6]:
def get_all_cars_in_grid(puzzle:np.array):
    
    #find horizontal cars
    cars = []
    for row in puzzle[range(6)]:
        #https://stackoverflow.com/a/6352456 for finding consecutive duplicates in a list
        grouped_row = [(letter, sum(1 for i in g)) for letter, g in groupby(row)]
        grouped_row = [Car(True, letter, num, np.where(puzzle==letter))
                       for letter, num in grouped_row if letter!='.' and num>1]
        if grouped_row is not None:
            cars = cars + grouped_row
    #find vertical cars
    for i in range(6):
        column = puzzle[:,i]
        grouped_column = [(letter, sum(1 for i in g)) for letter, g in groupby(column)]
        grouped_column = [Car(False, letter, num, np.where(puzzle==letter))
                          for letter, num in grouped_column if letter!='.' and num>1]
        if grouped_column is not None:
            cars = cars + grouped_column
    return cars

In [7]:
cars = get_all_cars_in_grid(puzzle_grid)

In [8]:
for car in cars:
    print(car)

Horizontal car 'B' with a length of 2. Indices i:[0 0] and j:[0 1]
Horizontal car 'C' with a length of 2. Indices i:[1 1] and j:[4 5]
Horizontal car 'A' with a length of 2. Indices i:[2 2] and j:[3 4]
Horizontal car 'D' with a length of 2. Indices i:[3 3] and j:[1 2]
Horizontal car 'F' with a length of 2. Indices i:[5 5] and j:[2 3]
Vertical car 'G' with a length of 3. Indices i:[3 4 5] and j:[0 0 0]
Vertical car 'H' with a length of 2. Indices i:[4 5] and j:[1 1]
Vertical car 'I' with a length of 3. Indices i:[0 1 2] and j:[2 2 2]
Vertical car 'J' with a length of 2. Indices i:[0 1] and j:[3 3]
Vertical car 'K' with a length of 2. Indices i:[3 4] and j:[3 3]
Vertical car 'L' with a length of 2. Indices i:[4 5] and j:[4 4]
Vertical car 'M' with a length of 2. Indices i:[2 3] and j:[5 5]


1. Group the consecutive duplicate letters
2. Find the neighbours of the car's letter.
3. Append a tuple of the group of empty spaces with the array index adjacent to the car to a list if it neighbours the car
4. Return this list

Front spaces are defined as j+1 or i+1. AKA going down or going to the right of the matrix.
Back spaces are defined as j-1 or i-1. AKA going up or going to the left of the matrix.

In [9]:
def find_available_spaces_for_car_on_grid(car:Car, puzzle_grid:np.array):
    
    back_spaces = 0
    front_spaces = 0
    
    if car.horizontal:
        relevant_groups = [(letter, sum(1 for i in g)) for letter, g in groupby(puzzle_grid[car.arr_indices[0][1],:])]
        index_of_car=[i for i, group in enumerate(relevant_groups) if group[0]==car.letter][0]
    else:
        relevant_groups = [(letter, sum(1 for i in g)) for letter, g in groupby(puzzle_grid[:,car.arr_indices[1][0]])]
        index_of_car=[i for i, group in enumerate(relevant_groups) if group[0]==car.letter][0]
    
    try:
        front_neighbour = relevant_groups[index_of_car+1]
        front_spaces = front_neighbour[1] if front_neighbour[0]=='.' else 0
    except IndexError:
        front_spaces = 0
    try:
        back_neighbour = relevant_groups[index_of_car-1] if index_of_car > 0 else ('',0)
        back_spaces = back_neighbour[1] if back_neighbour[0]=='.' else 0
    except IndexError:
        back_spaces = 0
    
    return car.letter, back_spaces, front_spaces

In [10]:
car_moves = [find_available_spaces_for_car_on_grid(car, puzzle_grid) for car in cars]
print(car_moves)

[('B', 0, 0), ('C', 0, 0), ('A', 0, 0), ('D', 0, 0), ('F', 0, 0), ('G', 2, 0), ('H', 0, 0), ('I', 0, 0), ('J', 0, 0), ('K', 0, 0), ('L', 1, 0), ('M', 0, 2)]


1. Pass car_moves into move_car? Let's assume that the passed amount is valid for the puzzle
2. Update the car's array indices
3. Update the grid/empty space, replace the car's old position with '.'s, then place the car at its new position
4. return new puzzle grid

In [11]:
def move_car(car:Car, puzzle_grid:np.array, amount:int):
    car.update_arr_indices(amount)
    new_grid = puzzle_grid
    new_grid[new_grid==car.letter] = '.'
    for x in range(len(car.arr_indices[0])):
        new_grid[car.arr_indices[0][x], car.arr_indices[1][x]] = car.letter
    return new_grid

In [12]:
new_grid = move_car(cars[5], puzzle_grid, -2)

In [13]:
new_grid

array([['B', 'B', 'I', 'J', '.', '.'],
       ['G', '.', 'I', 'J', 'C', 'C'],
       ['G', '.', 'I', 'A', 'A', 'M'],
       ['G', 'D', 'D', 'K', '.', 'M'],
       ['.', 'H', '.', 'K', 'L', '.'],
       ['.', 'H', 'F', 'F', 'L', '.']], dtype='<U1')